In [1]:
import tensorflow as tf
import torch

# check if GPU is being used
print("GPU Status: ", end = '')
if len(tf.config.list_physical_devices('GPU')) != 0: print("Active")
else: print("Inactive")

GPU Status: Active


In [2]:
# TensorFlow and tf.keras
import numpy as np
from tensorflow import keras
import autokeras as ak

#tf version should be 2.5 or higher
tf.__version__

Using TensorFlow backend


'2.10.0'

In [3]:
#get data
(train_images, train_labels), (test_images, test_labels) = \
      keras.datasets.fashion_mnist.load_data()

In [4]:
#scale data
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
#reshape inputs for CNN layers
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [6]:
import os
from tensorboard.plugins.hparams import api as hp

# set hparams
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'adagrad', 'rmsprop', 'sgd']))
HP_CONV_ACTIVATION = hp.HParam('conv_activation', hp.Discrete(['relu', 'elu', 'gelu']))
HP_DENSE_ACTIVATION = hp.HParam('dense_activation', hp.Discrete(['relu', 'elu', 'gelu']))


In [7]:
def train_model(hparams):

  model = keras.Sequential([
    keras.layers.Conv2D(kernel_size = 3, filters = 12, use_bias = False, padding = 'same', input_shape = (28, 28, 1)),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation(hparams[HP_CONV_ACTIVATION]),

    keras.layers.Conv2D(kernel_size = 6, filters = 24, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Flatten(),

    keras.layers.Dense(200, use_bias = False),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation(hparams[HP_DENSE_ACTIVATION]),

    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
  _, accuracy = model.evaluate(test_images, test_labels)
  return accuracy


In [8]:
conv_activation_name = []
dense_activation_name = []
activation_accuracy = []

for conv_activation in HP_CONV_ACTIVATION.domain.values:
    for dense_activation in HP_DENSE_ACTIVATION.domain.values:
        hparams = { 
            HP_CONV_ACTIVATION: conv_activation,
            HP_DENSE_ACTIVATION: dense_activation,
        }
        accuracy = train_model(hparams)
        conv_activation_name.append(conv_activation)
        dense_activation_name.append(dense_activation)
        activation_accuracy.append(accuracy)

Epoch 1/10
1875/1875 [==============================] - 16s 7ms/step - loss: 0.3819 - accuracy: 0.8643 - val_loss: 0.2904 - val_accuracy: 0.8980
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2480 - accuracy: 0.9111 - val_loss: 0.2338 - val_accuracy: 0.9157
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2098 - accuracy: 0.9234 - val_loss: 0.2303 - val_accuracy: 0.9142
Epoch 4/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1790 - accuracy: 0.9340 - val_loss: 0.2183 - val_accuracy: 0.9214
Epoch 5/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1494 - accuracy: 0.9452 - val_loss: 0.2196 - val_accuracy: 0.9228
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1233 - accuracy: 0.9545 - val_loss: 0.2447 - val_accuracy: 0.9195
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1009 - accuracy: 0.9625 - val_loss: 0.2413 - val_ac

In [13]:
for i in range (9):
        print("---------------------------------------------------------------------------")
        print("CONV Layer Activation: " + str(conv_activation_name[i]))
        print("DENSE Layer Activation: " + str(dense_activation_name[i]))
        print("Accuracy: " + str(activation_accuracy[i]))
        print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
CONV Layer Activation: elu
DENSE Layer Activation: elu
Accuracy: 0.9259999990463257
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Activation: elu
DENSE Layer Activation: gelu
Accuracy: 0.9279999732971191
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Activation: elu
DENSE Layer Activation: relu
Accuracy: 0.9272000193595886
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Activation: gelu
DENSE Layer Activation: elu
Accuracy: 0.921999990940094
---------------------------------------------------------------------------
------------------------------------------------------